In [2]:
import graphlab

In [3]:
sales = graphlab.SFrame('kc_house_data.gl')

[INFO] graphlab.cython.cy_server: GraphLab Create v2.1 started. Logging: /tmp/graphlab_server_1539313465.log


This non-commercial license of GraphLab Create for academic use is assigned to r38411@yahoo.com and will expire on August 31, 2019.


In [4]:
graphlab.canvas.set_target('ipynb')
sales.print_rows(num_rows=3)

+------------+---------------------------+----------+----------+-----------+
|     id     |            date           |  price   | bedrooms | bathrooms |
+------------+---------------------------+----------+----------+-----------+
| 7129300520 | 2014-10-13 00:00:00+00:00 | 221900.0 |   3.0    |    1.0    |
| 6414100192 | 2014-12-09 00:00:00+00:00 | 538000.0 |   3.0    |    2.25   |
| 5631500400 | 2015-02-25 00:00:00+00:00 | 180000.0 |   2.0    |    1.0    |
+------------+---------------------------+----------+----------+-----------+
+-------------+----------+--------+------------+------+-----------+-------+------------+
| sqft_living | sqft_lot | floors | waterfront | view | condition | grade | sqft_above |
+-------------+----------+--------+------------+------+-----------+-------+------------+
|    1180.0   |   5650   |   1    |     0      |  0   |     3     |   7   |    1180    |
|    2570.0   |   7242   |   2    |     0      |  0   |     3     |   7   |    2170    |
|    770.0    | 

In [6]:
train_data, test_data = sales.random_split(.8, seed=0)

In [21]:
def simple_linear_regression(input_feature, output):
    x = graphlab.SArray(input_feature)
    y = graphlab.SArray(output)
    N = len(x)
    slope = 1.0 * (sum(x*y)-sum(x)*sum(y)*1.0/N)/(sum(x*x)-sum(x)*sum(x)*1.0/N)
    intercept = y.mean() - slope*x.mean()
    return (intercept, slope)

In [9]:
input_feature = train_data['sqft_living']
output = train_data['price']
(squarefeet_intercept, squarefeet_slope) = simple_linear_regression(input_feature, output)
print squarefeet_intercept, squarefeet_slope

-47116.0765749 281.958838568


# Question 3:
# squarefeet_intercept = -47116.07, squarefeet_slope = 281.958

In [10]:
## Question 5:

In [11]:
def get_regression_predictions(input_feature, intercept, slope):
    x = graphlab.SArray(input_feature)
    y = intercept + slope * x
    return y

In [23]:
def test_func():
    i_feature = [0,1,2,3,4]
    out = [1,3,7,13,21]
    (w0, w1) = simple_linear_regression(i_feature, out)
    print w0, w1
    my_output = get_regression_predictions(i_feature, w0, w1)
    print my_output
test_func()

-1.0 5.0
[-1.0, 4.0, 9.0, 14.0, 19.0]
